In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup

In [27]:
df = pd.read_excel('/Users/steven/Desktop/微创/医院主数据20240520.xlsx',header=1)
df.head(5)

,医院编码,医院名称,别名,目标医院编码,目标医院名称,分级,分等,省,市,区,经营模式,类别,地址
0,110101001,北京市第六医院,NaN,110101001,北京市第六医院,二级,甲,北京市,北京城区,东城区,公立,综合,北京市东城区交道口北二条36号
1,110101002,北京市公安医院,NaN,110101002,北京市公安医院,三级,甲,北京市,北京城区,东城区,公立,综合,北京市东城区银闸胡同25号
2,110101003,北京市和平里医院,NaN,110101003,北京市和平里医院,二级,甲,北京市,北京城区,东城区,公立,综合,北京市东城区和平里北街18号、东城区和平里西街19号楼一层12307号
3,110101004,北京市普仁医院,NaN,110101004,北京市普仁医院,二级,甲,北京市,北京城区,东城区,公立,综合,北京市崇文区崇外大街100号
4,110101005,中国医学科学院北京协和医院,NaN,110101005,中国医学科学院北京协和医院,三级,甲,北京市,北京城区,东城区,公立,综合,东城区东单帅府园1号(东院);西城区大木仓胡同41号(西院)


In [28]:
huanan_df = df.loc[df['省'].isin(['广东省', '广西壮族自治区', '海南省'])].reset_index(drop=True)
huanan_df

,医院编码,医院名称,别名,目标医院编码,目标医院名称,分级,分等,省,市,区,经营模式,类别,地址
0,440100696,广州中兴运动损伤专科医院,NaN,440100696,广州中兴运动损伤专科医院,二级,丙,广东省,广州市,白云区,民营,NaN,广州市白云区机场路松云街39号
1,440101001,广州开发区医院,NaN,440101001,广州开发区医院,三级,甲,广东省,广州市,黄埔区,公立,综合,广州市黄埔区开发区友谊路196号
2,440101002,广州市南沙中心医院,NaN,440101002,广州市南沙中心医院,三级,甲,广东省,广州市,南沙区,公立,NaN,广州市南沙开发区丰泽路105号
3,440101003,广州医科大学附属第一医院,NaN,440101003,广州医科大学附属第一医院,三级,甲,广东省,广州市,越秀区,公立,综合,广东省广州市越秀区沿江西路151号
4,440101004,南方医科大学南方医院,NaN,440101004,南方医科大学南方医院,三级,甲,广东省,广州市,白云区,公立,综合,广东省广州市广州大道北1838号
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6019,469055491,陵水黎族自治县第二人民医院,NaN,469055491,陵水黎族自治县第二人民医院,二级,甲,海南省,陵水黎族自治县,陵水黎族自治县,公立,NaN,香水湾大道 16 号
6020,469068601,保亭县人民医院,NaN,469068601,保亭县人民医院,二级,甲,海南省,保亭黎族苗族自治县,保亭黎族苗族自治县,公立,NaN,海南省保亭黎族苗族自治县保城镇新兴东路58号
6021,469072568,万宁市人民医院,NaN,469072568,万宁市人民医院,二级,甲,海南省,万宁市,万宁市,公立,NaN,海南省万宁市环市三东路1号
6022,469085446,临高县中医院,NaN,469085446,临高县中医院,二级,甲,海南省,临高县,临高县,公立,NaN,临城镇临城解放路


In [29]:
hos_names = huanan_df['医院名称']

In [2]:
df = pd.read_excel('目标医院.xlsx')
hos_names = df['医院名称']

In [3]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}
hdf_error=[]
keshi_error=[]
response_error=[]
hospital_error = []
hospital_df=pd.DataFrame(columns=['医院名称','别名','科室','分级','分等','经营模式','类别','地址','来源'])
for name in hos_names:
    hdf_url = 'https://so.haodf.com/index/search?type=&kw='+name
    hdf_response = requests.get(hdf_url, headers=headers)
    if hdf_response.status_code == 200:
        hdf_soup = BeautifulSoup(hdf_response.content, 'html.parser')
        div_elements = hdf_soup.find('div', class_="js-search-list")
        try:
            hospital_url = div_elements.find('a')['href']
            hospital_response = requests.get(hospital_url, headers=headers)
            if hospital_response.status_code == 200:
                hospital_soup = BeautifulSoup(hospital_response.content, 'html.parser')
                hospital_name = hospital_soup.find('h1', class_="i-other").text
                hospital_attributes = hospital_soup.find_all('span',class_='hospital-label-item')
                hospital_business_type = hospital_attributes[0].text
                hospital_level = hospital_attributes[1].text
                if '三' in hospital_level or '二' in hospital_level or '一' in hospital_level:
                    hospital_ji = hospital_level[0]+'级'
                else:
                    hospital_ji = None
                if '甲' in hospital_level or '乙' in hospital_level or '丙' in hospital_level:
                    hospital_deng = hospital_level[1]+'等'
                else:
                    hospital_deng = None
                hospital_category = hospital_attributes[2].text
                hospital_address = hospital_soup.find('div', class_='hos-address').text.replace("地址：", "")
                #科室
                keshi_url = div_elements.find('a')['href'].replace('.html','/keshi/list.html')
                keshi_response = requests.get(keshi_url, headers=headers)
                if keshi_response.status_code == 200:
                    keshi_soup = BeautifulSoup(keshi_response.content, 'html.parser')
                    keshi_elements = keshi_soup.find_all('div', class_="name-txt")
                    keshi_texts = [element.get_text(strip=True) for element in keshi_elements]
                    keshi_list = pd.DataFrame(keshi_texts,columns = ['科室'])
                    for keshi in keshi_list['科室']:
                        hospital_df.loc[len(hospital_df)] = [name,hospital_name, keshi, hospital_ji,hospital_deng,hospital_business_type,hospital_category,hospital_address, '好大夫']
                else:
                    keshi_error.append(name)
            else:
                hospital_error.append(name)
        except:
            response_error.append(name)
    else: 
        hdf_error.append(name)

#### response_error

In [4]:
hospital_error

[]

In [5]:
keshi_error

[]

In [6]:
hdf_error

[]

In [7]:
hos_names

0             广州开发区医院
1           广州市南沙中心医院
2        广州医科大学附属第一医院
3          南方医科大学南方医院
4       中国人民解放军第四二一医院
            ...      
516           澄迈县人民医院
517    昌江黎族自治县中西医结合医院
518           东方市人民医院
519           保亭县人民医院
520           万宁市人民医院
Name: 医院名称, Length: 521, dtype: object

In [32]:
hospital_df.to_excel('华南数据带属性.xlsx',index=False)